## Florida Department of Corrections Inmate Crime and Tattoo Database
#### Joey, Matt, Sean

### 1, 2. The Dataset, Downloaded

In [1]:
import pandas
import numpy as np
import pandas_access
DATASET = "./FDOC_January_2017.mdb" # From http://www.dc.state.fl.us/pub/obis_request.html 
                                    # ~1.3 gb; publicly available data.

### 3. Loading into a DF

In [2]:
dfs = {}
for tablename in pandas_access.list_tables(DATASET):
    dfs[tablename] = pandas_access.read_table(DATASET, tablename, low_memory = False)

### 4. Identifying Variables of Interest

In [3]:
dfs.keys()

[u'INMATE_RELEASE_INCARHIST',
 u'INMATE_ACTIVE_ROOT',
 u'INMATE_RELEASE_DETAINERS',
 u'OFFENDER_OFFENSES_CCS',
 u'INMATE_RELEASE_OFFENSES_CPS',
 u'INMATE_RELEASE_RESIDENCE',
 u'INMATE_RELEASE_SCARSMARKS',
 u'INMATE_ACTIVE_ALIASES',
 u'INMATE_ACTIVE_DETAINERS',
 u'INMATE_RELEASE_OFFENSES_prpr',
 u'INMATE_ACTIVE_SCARSMARKS',
 u'OFFENDER_ROOT',
 u'OFFENDER_RESIDENCE',
 u'INMATE_ACTIVE_OFFENSES_prpr',
 u'OFFENDER_ALIASES',
 u'INMATE_RELEASE_ROOT',
 u'INMATE_ACTIVE_INCARHIST',
 u'INMATE_ACTIVE_OFFENSES_CPS',
 u'CONTENTS',
 u'INMATE_RELEASE_ALIASES']

We initially planned to duplicate the experiment that linked tattoos and crimes, so we'll look at `SCARSMARKS` and `OFFENSES`.

### 5. Validation and Plotting

In [4]:
# http://www.dc.state.fl.us/pub/obis_request.html
# http://cdn.static-economist.com/sites/default/files/images/print-edition/20161224_XMC337_0.png

First, let's look at the number and types of tattoos:

In [5]:
tattoos = pandas.concat((dfs["INMATE_ACTIVE_SCARSMARKS"], dfs["INMATE_RELEASE_SCARSMARKS"]))
tattoo_descriptions = tattoos.Description
len(tattoo_descriptions)

1266840

In [6]:
tattoo_types = {
    "face": ["FACE", "HEAD", "CHEEK", "CHIN"],
    "white supremacist": ["WHITE", "ARYAN", "NAZI", "SWASTIKA", "14", "88"],
    "three dots": ["3 DOTS", "THREE DOTS", "THREEDOTS"],
    "tear drop": ["TEAR DROP", "TEARDROP"],
    "guns": ["GUN", "RIFLE", "BULLET"],
    "laugh now, cry later": ["LAUGH NOW", "CRY LATER", "LAUGHNOW", "CRYLATER"],
    "christian": ["CROSS", "HOLY", "ANGEL", "JESUS", "PRAY"],
    "satanic": ["PENTA", "DEVIL", "DEMON", "BAPHOMET"],
    "none": ["nan", "NaN"]
}
for category, keywords in tattoo_types.items():
    counts = tattoo_descriptions.str.contains("|".join(keywords)).value_counts()
    count = counts[True] if True in counts else 0
    print("{0: <20} {1: >7}".format(category, count))

none                       0
christian             138926
three dots              2753
satanic                18522
tear drop               7946
laugh now, cry later    4397
guns                   21472
white supremacist       6745
face                   71814


Next, let's look at the number and types of crimes:

In [7]:
crimes = pandas.concat((dfs["INMATE_ACTIVE_OFFENSES_prpr"], dfs["INMATE_ACTIVE_OFFENSES_CPS"], dfs["INMATE_RELEASE_OFFENSES_prpr"], dfs["INMATE_RELEASE_OFFENSES_CPS"]))
crime_descriptions = crimes.adjudicationcharge_descr
len(crime_descriptions)

2385493

In [8]:
crime_types = {
    "drugs": ["DRUG", "COCAINE", "SUBS", "MARIJUANA"],
    "theft": ["BURG", "THEFT", "THFT"],
    "robbery": ["ROBB"],
    "assault": ["ASLT", "ASSLT", "ASSAULT", "BATT"],
    "murder": ["MUR", "MRDR"]
}
for category, keywords in crime_types.items():
    counts = crime_descriptions.str.contains("|".join(keywords)).value_counts()
    count = counts[True] if True in counts else 0
    print("{0: <7} {1: >6}".format(category, count))

drugs   427856
assault 206298
murder   33311
robbery 140578
theft   720649


Now, let's group them together and use the totals in each category to calculate a ratio:

In [9]:
tattoos_crimes = pandas.merge(tattoos, crimes, "outer", "DCNumber")
tattoos_crimes.rename(index = str, columns = {
        "Description": "tattoos",
        "adjudicationcharge_descr": "crimes"
    }, inplace = True)

In [10]:
grouped_tattoos = tattoos_crimes.tattoos.groupby(tattoos_crimes.DCNumber).apply(lambda x: ",".join(set(x.astype(str))))
grouped_crimes = tattoos_crimes.crimes.groupby(tattoos_crimes.DCNumber).apply(lambda x: ",".join(set(x.astype(str))))
tattoos_and_crimes = pandas.concat([grouped_tattoos, grouped_crimes], axis = 1).reset_index()

In [11]:
tattoo_totals = {}
for tattoo, tattoowords in tattoo_types.items():
    tattoo_totals[tattoo] = tattoo_totals.get(tattoo, 0) + \
                            len(tattoos_and_crimes[tattoos_and_crimes.tattoos.str.contains("|".join(tattoowords))])
tattoo_totals

{'christian': 98267,
 'face': 51337,
 'guns': 17048,
 'laugh now, cry later': 3860,
 'none': 149392,
 'satanic': 14919,
 'tear drop': 7446,
 'three dots': 2539,
 'white supremacist': 5553}

In [12]:
crime_totals = {}
for crime, crimewords in crime_types.items():
    crime_totals[crime] = crime_totals.get(crime, 0) + \
                          len(tattoos_and_crimes[tattoos_and_crimes.crimes.str.contains("|".join(crimewords))])
crime_totals

{'assault': 124886,
 'drugs': 161320,
 'murder': 25788,
 'robbery': 75437,
 'theft': 193781}

In [13]:
groupings = {}
for tattoo, tattoowords in tattoo_types.items():
    for crime, crimewords in crime_types.items():
        count = len(tattoos_and_crimes[tattoos_and_crimes.tattoos.str.contains("|".join(tattoowords)) &\
                                       tattoos_and_crimes.crimes.str.contains("|".join(crimewords))])
        percent = 100.0 * count / crime_totals[crime]
        groupings[(tattoo, crime)] = (count, percent)
        print("{0: <20} tattoos, {1: <7} crimes: {2: >6} people ({3: >6.2f}% of like crimes)".format(tattoo, crime, count, percent))

none                 tattoos, drugs   crimes:  51838 people ( 32.13% of like crimes)
none                 tattoos, assault crimes:  39352 people ( 31.51% of like crimes)
none                 tattoos, murder  crimes:   9707 people ( 37.64% of like crimes)
none                 tattoos, robbery crimes:  20297 people ( 26.91% of like crimes)
none                 tattoos, theft   crimes:  54479 people ( 28.11% of like crimes)
christian            tattoos, drugs   crimes:  35032 people ( 21.72% of like crimes)
christian            tattoos, assault crimes:  27839 people ( 22.29% of like crimes)
christian            tattoos, murder  crimes:   4701 people ( 18.23% of like crimes)
christian            tattoos, robbery crimes:  17438 people ( 23.12% of like crimes)
christian            tattoos, theft   crimes:  46002 people ( 23.74% of like crimes)
three dots           tattoos, drugs   crimes:    598 people (  0.37% of like crimes)
three dots           tattoos, assault crimes:    769 people (  0.

### 6. Cleaning Issues

For validation, it's hard to know what the baseline that The Economist used was; are their relationships based on the typical crimes for someone with certain tattoos, or the typical tattoos had by someone who commits certain crimes, or something else entirely? We believe their actual baseline was inmates without tattoos, but it's hard to say.